In [1]:
# Dependencies
import pandas as pd
from sqlalchemy import create_engine,inspect,extract
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import datetime as dt
from datetime import timedelta
import numpy as np

In [2]:
Accidents_df = pd.read_csv("USAccidents_Clean.csv")
Accidents_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52690 entries, 0 to 52689
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 52690 non-null  object 
 1   Severity           52690 non-null  int64  
 2   Start_Time         52690 non-null  object 
 3   End_Time           52690 non-null  object 
 4   Start_Lat          52690 non-null  float64
 5   Start_Lng          52690 non-null  float64
 6   Street             52690 non-null  object 
 7   City               52690 non-null  object 
 8   State              52690 non-null  object 
 9   Weather_Condition  52137 non-null  object 
 10  Wind_Speed(mph)    50877 non-null  float64
 11  Precipitation(in)  51004 non-null  float64
 12  Amenity            52690 non-null  bool   
 13  Bump               52690 non-null  bool   
 14  Crossing           52690 non-null  bool   
 15  Give_Way           52690 non-null  bool   
 16  Junction           526

In [3]:
Accidents_df.head()

,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Street,City,State,Weather_Condition,...,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Time,StartDate,Start_year
0,A-2953990,2,2020-11-28 01:33:00,2020-11-28 03:13:16,33.963490,-118.370694,S La Cienega Blvd,Los Angeles,CA,Fair,...,False,False,False,True,False,True,False,01:33:00,2020-11-28,2020
1,A-2953993,2,2020-11-07 12:05:00,2020-11-07 16:03:00,32.917963,-96.856461,I-635 W,Dallas,TX,Fair,...,False,False,False,False,False,False,False,12:05:00,2020-11-07,2020
2,A-2953994,2,2020-11-03 17:10:08,2020-11-03 18:00:58,32.921262,-96.848712,I-635 W,Dallas,TX,Fair,...,False,False,False,False,False,False,False,17:10:08,2020-11-03,2020
3,A-2954010,2,2020-11-10 02:32:00,2020-11-10 04:46:20,34.152964,-118.278776,CA-134 E,Los Angeles,CA,Fair,...,False,False,False,False,False,False,False,02:32:00,2020-11-10,2020
4,A-2954063,2,2020-12-23 05:22:00,2020-12-23 11:31:41,34.071218,-118.235087,Stadium Way,Los Angeles,CA,Haze,...,False,False,False,False,False,False,False,05:22:00,2020-12-23,2020


In [4]:
pd.set_option('display.max_columns', None)

In [5]:
Accidents_df['StartDate']=pd.to_datetime(Accidents_df['StartDate'])
#Accidents_df['StartDate']

In [6]:
Accidents_df['Start_Time']=pd.to_datetime(Accidents_df['Start_Time'],format='%Y/%m/%d %H:%M:%S',errors='coerce')

In [7]:
Accidents_df['End_Time']=pd.to_datetime(Accidents_df['End_Time'],format='%Y/%m/%d %H:%M:%S',errors='coerce')

In [8]:
Accidents_df.drop(columns=['Wind_Speed(mph)', 'Precipitation(in)'],inplace=True)

In [9]:
Accidents_df.columns

Index(['ID', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng',
       'Street', 'City', 'State', 'Weather_Condition', 'Amenity', 'Bump',
       'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout',
       'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
       'Time', 'StartDate', 'Start_year'],
      dtype='object')

In [10]:
Accidents_df.rename(columns={"ID":"Accident_id",
                            "Start_year":"Year"},inplace=True)

In [11]:
Accidents_df.head()

,Accident_id,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Street,City,State,Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Time,StartDate,Year
0,A-2953990,2,2020-11-28 01:33:00,2020-11-28 03:13:16,33.963490,-118.370694,S La Cienega Blvd,Los Angeles,CA,Fair,False,False,False,False,False,False,False,False,False,True,False,True,False,01:33:00,2020-11-28,2020
1,A-2953993,2,2020-11-07 12:05:00,2020-11-07 16:03:00,32.917963,-96.856461,I-635 W,Dallas,TX,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,12:05:00,2020-11-07,2020
2,A-2953994,2,2020-11-03 17:10:08,2020-11-03 18:00:58,32.921262,-96.848712,I-635 W,Dallas,TX,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,17:10:08,2020-11-03,2020
3,A-2954010,2,2020-11-10 02:32:00,2020-11-10 04:46:20,34.152964,-118.278776,CA-134 E,Los Angeles,CA,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,02:32:00,2020-11-10,2020
4,A-2954063,2,2020-12-23 05:22:00,2020-12-23 11:31:41,34.071218,-118.235087,Stadium Way,Los Angeles,CA,Haze,False,False,False,False,False,False,False,False,False,False,False,False,False,05:22:00,2020-12-23,2020


In [12]:
Accidents_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52690 entries, 0 to 52689
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Accident_id        52690 non-null  object        
 1   Severity           52690 non-null  int64         
 2   Start_Time         52690 non-null  datetime64[ns]
 3   End_Time           52690 non-null  datetime64[ns]
 4   Start_Lat          52690 non-null  float64       
 5   Start_Lng          52690 non-null  float64       
 6   Street             52690 non-null  object        
 7   City               52690 non-null  object        
 8   State              52690 non-null  object        
 9   Weather_Condition  52137 non-null  object        
 10  Amenity            52690 non-null  bool          
 11  Bump               52690 non-null  bool          
 12  Crossing           52690 non-null  bool          
 13  Give_Way           52690 non-null  bool          
 14  Juncti

In [13]:
Weather_df = pd.read_csv("WeatherData.csv")
Weather_df.head()

,ID,Severity,Weather_Condition,City,State,Start_Time,Wind_Speed(mph),Precipitation(in),Year,AccYear
0,A-2953990,2,Fair,Los Angeles,CA,2020-11-28 01:33:00,0.0,0.0,2020-11-28,2020
1,A-2953993,2,Fair,Dallas,TX,2020-11-07 12:05:00,7.0,0.0,2020-11-07,2020
2,A-2953994,2,Fair,Dallas,TX,2020-11-03 17:10:08,12.0,0.0,2020-11-03,2020
3,A-2954010,2,Fair,Los Angeles,CA,2020-11-10 02:32:00,0.0,0.0,2020-11-10,2020
4,A-2954063,2,Haze,Los Angeles,CA,2020-12-23 05:22:00,3.0,0.0,2020-12-23,2020


In [14]:
Weather_df.rename(columns={"ID":"Accident_id",
                            "Precipitation(in)":"Precipitation",
                          "Wind_Speed(mph)":"Wind_Speed",
                          "Year":"StartDate",
                          "AccYear":"Year"},inplace=True)

In [15]:
Weather_df['Year']=Weather_df['Year'].astype(int)
Weather_df['StartDate']=pd.to_datetime(Weather_df['StartDate'])

In [16]:
Weather_df['Start_Time']=pd.to_datetime(Weather_df['Start_Time'],format='%Y-%m-%d %H:%M:%S',errors='coerce')

In [17]:
Weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52690 entries, 0 to 52689
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Accident_id        52690 non-null  object        
 1   Severity           52690 non-null  int64         
 2   Weather_Condition  52137 non-null  object        
 3   City               52690 non-null  object        
 4   State              52690 non-null  object        
 5   Start_Time         52690 non-null  datetime64[ns]
 6   Wind_Speed         50877 non-null  float64       
 7   Precipitation      51004 non-null  float64       
 8   StartDate          52690 non-null  datetime64[ns]
 9   Year               52690 non-null  int64         
dtypes: datetime64[ns](2), float64(2), int64(2), object(4)
memory usage: 4.0+ MB


### uploading dataframe to sqlite database

In [18]:
from sqlalchemy import Column, Integer, String, Float,BigInteger,Boolean
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import func
Base=declarative_base()

In [19]:
class USAccidents(Base):
    __tablename__='us_accidents'
    id = Column(Integer, primary_key=True)
    Accident_id= Column(String)
    Severity= Column(Integer)
    Start_Time=Column(String)
    End_Time= Column(String)
    Start_Lat= Column(Float)
    Start_Lng= Column(Float)
    Street=Column(String)
    City =Column(String)
    State= Column(String)
    Weather_Condition=Column(String)
    Amenity= Column(Boolean)
    Bump=Column(Boolean) 
    Crossing= Column(Boolean)
    Give_Way=Column(Boolean)
    Junction=Column(Boolean) 
    No_Exit =Column(Boolean)
    Railway =Column(Boolean)
    Roundabout=Column(Boolean) 
    Station =Column(Boolean)
    Stop =Column(Boolean)
    Traffic_Calming=Column(Boolean)
    Traffic_Signal= Column(Boolean)
    Turning_Loop =Column(Boolean)
    Time= Column(String)
    StartDate=Column(String)
    Year= Column(Integer)

In [20]:
class USWeather(Base):
    __tablename__='us_weather'
    id = Column(Integer, primary_key=True)
    Accident_id = Column(String)
    Severity=Column(Integer)
    Weather_Condition =Column(String)         
    Precipitation= Column(Float)             
    Wind_Speed = Column(Float)               
    Start_Time = Column(String)         
    City = Column(String)                     
    State = Column(String)                    
    StartDate = Column(String)         
    Year =Column(Integer)              
    
    

In [21]:
engine = create_engine('sqlite:///us_accidents.db', echo=False)
conn=engine.connect()
Base.metadata.create_all(engine)

In [22]:
Accidents_df.to_sql('us_accidents', con=engine,if_exists="append",index=False)
Base.metadata.create_all(engine)

In [23]:
Weather_df.to_sql('us_weather', con=engine,if_exists="append",index=False)
Base.metadata.create_all(engine)

In [24]:
engine = create_engine('sqlite:///us_accidents.db', echo=False)
conn=engine.connect()

In [25]:
#conn.execute("DROP TABLE us_accidents ")

In [26]:
#conn.execute("DROP TABLE us_weather ")

In [27]:
inspector = inspect(engine)

In [28]:
#conn.execute("DROP TABLE us_accidents ")

In [29]:
columns = inspector.get_columns('us_weather')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
Accident_id VARCHAR
Severity INTEGER
Weather_Condition VARCHAR
Precipitation FLOAT
Wind_Speed FLOAT
Start_Time VARCHAR
City VARCHAR
State VARCHAR
StartDate VARCHAR
Year INTEGER


In [30]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
base=automap_base()
base.prepare(engine, reflect=True)

In [31]:
base.classes.keys()

['us_accidents', 'us_weather']

In [32]:
#pd.read_sql_table('us_accidents',conn)

In [33]:
columns = inspector.get_columns('us_accidents')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
Accident_id VARCHAR
Severity INTEGER
Start_Time VARCHAR
End_Time VARCHAR
Start_Lat FLOAT
Start_Lng FLOAT
Street VARCHAR
City VARCHAR
State VARCHAR
Weather_Condition VARCHAR
Amenity BOOLEAN
Bump BOOLEAN
Crossing BOOLEAN
Give_Way BOOLEAN
Junction BOOLEAN
No_Exit BOOLEAN
Railway BOOLEAN
Roundabout BOOLEAN
Station BOOLEAN
Stop BOOLEAN
Traffic_Calming BOOLEAN
Traffic_Signal BOOLEAN
Turning_Loop BOOLEAN
Time VARCHAR
StartDate VARCHAR
Year INTEGER


In [34]:
session=Session(bind=engine)

In [35]:
## year filter query for leafletplot
year=2019
results=session.query(USAccidents).filter(USAccidents.Year==year).limit(10).all()
#print(results)
for result in results:
    print(result.City,result.State,result.Street,result.Start_Lat,result.Start_Lng,result.Severity,result.Weather_Condition)

Los Angeles CA N Spring St 34.056344 -118.241267 2 Fair
Los Angeles CA Golden State Fwy N 34.073060999999996 -118.21915600000001 2 Fair
Los Angeles CA  Whittier Blvd 34.016994 -118.14111499999999 2 Fair
Los Angeles CA Hollywood Fwy N 34.101681 -118.31649499999999 2 Fair
Los Angeles CA E Olympic Blvd 34.012465999999996 -118.137455 2 Fair
Los Angeles CA Golden State Fwy S 34.150413 -118.280699 2 Fair
Los Angeles CA S Los Angeles St 34.032086 -118.262248 2 Fair
San Diego CA Escondido Fwy S 32.806194 -117.11360800000001 2 Fair
Los Angeles CA Santa Ana Fwy N 34.059158000000004 -118.245397 2 Partly Cloudy
Dallas TX US-77 N 32.71821 -96.82807 3 Fair


In [36]:
start='2019-01-01'
end='2020-01-01'
start_date = dt.datetime.strptime(start, '%Y-%m-%d').date()
end_date=dt.datetime.strptime(end, '%Y-%m-%d').date()


results=session.query(USAccidents.City,func.count(USAccidents.Accident_id)).group_by(USAccidents.City).\
                    filter(USAccidents.StartDate>=start_date).filter(USAccidents.StartDate<=end).all()

print(results)

[('Dallas', 1560), ('Los Angeles', 9380), ('New York', 447), ('San Diego', 3516)]


In [37]:
#query for line plot with timedata
results=session.query(USAccidents.City,func.count(USAccidents.City),func.strftime("%Y-%m",USAccidents.StartDate)).\
                group_by(USAccidents.City,func.strftime("%Y-%m",USAccidents.StartDate)).limit(10).all()
print (results)

[('Dallas', 195, '2019-01'), ('Dallas', 156, '2019-02'), ('Dallas', 114, '2019-03'), ('Dallas', 169, '2019-04'), ('Dallas', 168, '2019-05'), ('Dallas', 177, '2019-06'), ('Dallas', 164, '2019-07'), ('Dallas', 127, '2019-08'), ('Dallas', 101, '2019-09'), ('Dallas', 96, '2019-10')]


In [38]:
# query for hourly accidents count in each city
results=session.query(USAccidents.City,func.count(USAccidents.Accident_id),func.strftime("%H",USAccidents.Start_Time)).\
                        group_by(USAccidents.City,func.strftime("%H",USAccidents.Start_Time)).limit(10).all()
print (results)



[('Dallas', 421, '00'), ('Dallas', 398, '01'), ('Dallas', 426, '02'), ('Dallas', 429, '03'), ('Dallas', 337, '04'), ('Dallas', 436, '05'), ('Dallas', 512, '06'), ('Dallas', 427, '07'), ('Dallas', 464, '08'), ('Dallas', 437, '09')]


In [39]:
#query for weekday wise accidents count in each city
results=session.query(USAccidents.City,func.count(USAccidents.Accident_id),func.strftime("%w",USAccidents.Start_Time)).\
                        group_by(USAccidents.City,func.strftime("%w",USAccidents.Start_Time)).limit(10).all()
print (results)

[('Dallas', 1301, '0'), ('Dallas', 1588, '1'), ('Dallas', 1673, '2'), ('Dallas', 1788, '3'), ('Dallas', 1596, '4'), ('Dallas', 1591, '5'), ('Dallas', 1246, '6'), ('Los Angeles', 3247, '0'), ('Los Angeles', 4480, '1'), ('Los Angeles', 4621, '2')]


In [40]:
results=session.query(USAccidents.Severity,func.count(USAccidents.Severity)).group_by(USAccidents.Severity).order_by(USAccidents.Severity).all()
print(results)

[(1, 6), (2, 48142), (3, 3412), (4, 1130)]


In [41]:
#query citywise yearly count
results=session.query(USAccidents.Year,func.count(USAccidents.Year),USAccidents.City).group_by(USAccidents.Year,USAccidents.City).all()
print(results)

[(2019, 1560, 'Dallas'), (2019, 9380, 'Los Angeles'), (2019, 447, 'New York'), (2019, 3516, 'San Diego'), (2020, 9223, 'Dallas'), (2020, 19930, 'Los Angeles'), (2020, 2104, 'New York'), (2020, 6530, 'San Diego')]


In [42]:
#query weather condition
results=session.query(USAccidents.Weather_Condition,USAccidents.Year,USAccidents.City,func.count(USAccidents.Year)).group_by(USAccidents.Year,USAccidents.City,USAccidents.Weather_Condition).limit(10).all()
print(results)

[(None, 2019, 'Dallas', 14), ('Clear', 2019, 'Dallas', 85), ('Cloudy', 2019, 'Dallas', 192), ('Cloudy / Windy', 2019, 'Dallas', 7), ('Drizzle', 2019, 'Dallas', 4), ('Fair', 2019, 'Dallas', 488), ('Fair / Windy', 2019, 'Dallas', 5), ('Fog', 2019, 'Dallas', 4), ('Haze', 2019, 'Dallas', 6), ('Heavy Rain', 2019, 'Dallas', 1)]


In [43]:
#query to find relatio among precipitation, windspeed, severity
results=session.query(USWeather.Precipitation,USWeather.Wind_Speed,USWeather.Severity).limit(10).all() 
print(results)

[(0.0, 0.0, 2), (0.0, 7.0, 2), (0.0, 12.0, 2), (0.0, 0.0, 2), (0.0, 3.0, 2), (0.0, 13.0, 2), (0.0, 5.0, 2), (0.0, 8.0, 2), (0.0, 6.0, 2), (0.0, 3.0, 2)]


In [44]:
#query for accidents due to different road condition in two years 

Bump_accidents=session.query(USAccidents.Year,func.count(USAccidents.Year)).group_by(USAccidents.Year).filter(USAccidents.Bump==True).all()
Junction_accidents=session.query(USAccidents.Year,func.count(USAccidents.Year)).group_by(USAccidents.Year).filter(USAccidents.Junction==True).all()
No_Exit_accidents=session.query(USAccidents.Year,func.count(USAccidents.Year)).group_by(USAccidents.Year).filter(USAccidents.No_Exit==True).all()
Railway_accidents=session.query(USAccidents.Year,func.count(USAccidents.Year)).group_by(USAccidents.Year).filter(USAccidents.Railway==True).all()
Station_accidents=session.query(USAccidents.Year,func.count(USAccidents.Year)).group_by(USAccidents.Year).filter(USAccidents.Station==True).all()
Stop_accidents=session.query(USAccidents.Year,func.count(USAccidents.Year)).group_by(USAccidents.Year).filter(USAccidents.Stop==True).all()
Traffic_Calming_accidents=session.query(USAccidents.Year,func.count(USAccidents.Year)).group_by(USAccidents.Year).filter(USAccidents.Traffic_Calming==True).all()
Traffic_Signal_accidents=session.query(USAccidents.Year,func.count(USAccidents.Year)).group_by(USAccidents.Year).filter(USAccidents.Traffic_Signal==True).all()

print(f'''Bump_accidents:{Bump_accidents},Junction_accidents:{Junction_accidents},No_Exit_accidents:{No_Exit_accidents},
Railway_accidents:{Railway_accidents},Station_accidents:{Station_accidents},
Stop_accidents:{Stop_accidents},Traffic_Calming_accidents:{Traffic_Calming_accidents},Traffic_Signal_accidents:{Traffic_Signal_accidents}''')



Bump_accidents:[(2019, 2), (2020, 6)],Junction_accidents:[(2019, 2379), (2020, 5468)],No_Exit_accidents:[(2019, 12), (2020, 131)],
Railway_accidents:[(2019, 256), (2020, 621)],Station_accidents:[(2019, 554), (2020, 1062)],
Stop_accidents:[(2019, 173), (2020, 313)],Traffic_Calming_accidents:[(2019, 2), (2020, 6)],Traffic_Signal_accidents:[(2019, 1667), (2020, 5414)]


In [45]:
#query for accidents in day and night time
# sqlalchemy import or_
#evestart=dt.datetime.strptime('17:00:00', '%H:%M:%S').time()
#midnight=dt.datetime.strptime('24:00:00', '%H:%M:%S').time()
#morningstart=dt.datetime.strptime('00:00:00', '%H:%M:%S').time()
#daystart=dt.datetime.strptime('05:00:00', '%H:%M:%S').time()

#day_accidents_count=session.query(USAccidents).filter(USAccidents.Time>daystart,USAccidents.Time<evestart).count